In [1]:
module QuantumRelay

using SymPy
using JuMP
using Clp
using IterTools
using LLLplus
using GSL
using Distributions
using LinearAlgebra
using SmithNormalForm
using SpecialFunctions

export
	qrelay_op,
	op_mat,
	scan_maker,
	nonneg,
	QRelaySampler

include("utility.jl")
include("operators.jl")
include("Clp.jl")
include("distributions.jl")

struct QRelaySampler
	prob::Function #return the probablility
	psetproposal::Function #return next combination of sample

    #input parameters:
    #mat: the matrix p_ij in the note (10)
    #coef: the coefficient c in the note (10)
    #omega: the coefficient A in the note (10)
    #pdet0: the probability of detection for each detector
	function QRelaySampler(mat::Array{T, 2}, coef, omega, pdet0) where T <: Int
		F=smith(mat)
		U=F.S
		V=F.T
		S=diagm(F)
        #the SmithNormalForm returns P=USV. Inverse the matrices so Ui/Vi is the same as U/V in the note (18)
		Ui = inv(U) 
		Vi = inv(V)
		s = diag(S)
		r = count(!iszero, s)
		s0 = s[1:r]
		@assert s0 == ones(r)
		ui1 = Ui[1:r, :]
		ui2 = Ui[r+1:end, :]
		vi1 = Vi[:, 1:r]
		vi2 = Vi[:, r+1:end]
		vi2 = lll(vi2)[1] #Lenstra–Lenstra–Lovász lattice basis reduction
		T0 = vi1*ui1
		ui2oc = orthocomp(ui2) #orhogonal complements
		setc, scan = scan_maker(vi2) #make the scanner for the algorithm1 in the note
        
        #compute the probability for an ideal system
        #na: the photon numbers in a output mode
		function prob(na)
		    @assert count(!iszero, ui2*na) == 0
		    b = T0*na
		    setc(-b)
		    total = 0.0
		    for x in Channel(scan)
		        nab = vi2*x + b #the photon numbers for each item in the sum in the note (10)
		        total += prod([c.^complex(n)/gamma(n+1) for (c, n) in zip(coef, nab)])
		    end
		    return abs2(total*omega)
		end

        #compute the probability of detection
        #q: the number of photons detectors report
        #na: the number of photons arrived at detector
        #mask: if there is no detector in this channel, mask=0
		function prob(q, na, mask)
		    q0 = round.(Int, q.>0)
		    m0 = round.(Int, mask)
		    return prod((q0 + (ones(length(q0))-2q0).*pdet0(na)).^m0)
		end

		psetproposal(x::Vector) = QuantumRelay.OrthoNNDist(x, ui2oc)

		new(prob, psetproposal)

	end
end
end

Main.QuantumRelay

In [2]:
using Plots
using Random
using SymPy
using CSV
using DataFrames
using Main.QuantumRelay
using Distributions

#calling the github code 
function quantumrelay(alpha,delta,name::String,n)   #n is the number of sources for 1 relay we have 2 sources
   
    chi = fill(sqrt(0.06), n)                  # the parameter chi 
    phi = im*tanh.(chi)
    omega = -log(prod(cosh.(chi)))
    syms, op = qrelay_op(n, phi, alpha, delta)
    op_a, op_ab, mat, coef = op_mat(op)

    op_q2 = [syms.apH[1], syms.apV[1], syms.bpH[end], syms.bpV[end]]
    op_q1 = [syms.apH[2:end]..., syms.apV[2:end]..., syms.bpH[1:end-1]..., syms.bpV[1:end-1]...]
    mask_q1 = [op in op_q1 for op in op_a];
    
    mask_q2 = [op in op_q2 for op in op_a];
    qq = [x in syms.apH || x in syms.bpV ? 1 : 0 for x in op_a]
           
    pdet0 = pdet_maker(0.04, 3e-5)
    qrs = QRelaySampler(mat, coef, omega, pdet0)
    targetcache=Dict{Vector{Int}, Float64}()
    target(x::Vector)= (qrs.prob(qq, x, mask_q1) * qrs.prob(x))      #the target function of MCMC
    Iteration=2^17
    burnin=2^10
    samples=Iteration+burnin
    step=5
    save_iter=burnin:samples
    dist= qrs.psetproposal           #the proposal distribution
    selected=Array[]
    Q = Float64[]
    
    current_x = zeros(8)
    @time for i in 2:samples            #from this line the MCMC algorithm starts
        
        proposed_x= rand(dist(current_x))
        
        prop_proposed= pdf(dist(current_x), proposed_x)
        
        prop_current= pdf(dist(proposed_x), current_x)
        
        
        A= min(1,(target(proposed_x)*prop_current)/(target(current_x)*prop_proposed))
        
        
        if rand() < A
            #if i in save_iter
                
                
            push!(selected, proposed_x)
            push!(Q, qrs.prob(qq, proposed_x, mask_q2))
            
            #end
    
            current_x = proposed_x
        else
            current_x= current_x
        end        
    
        
    end
    
    return selected, Q
end

quantumrelay (generic function with 1 method)

In [3]:
prob=[]
accepted=[]
for i = 0:14
    beta = i*pi/14
    name = string(i)
    selected, Q = quantumrelay(pi/4, beta, name,2)
    push!(accepted,length(selected))
    println("beta:", beta)
    push!(prob,Q)
    df=DataFrame(selected=selected)

end

2479.428721 seconds (738.27 M allocations: 57.862 GiB, 0.54% gc time, 0.32% compilation time)
beta:0.0


LoadError: TaskFailedException

[91m    nested task error: [39mAssertionError: MOI.get(m, MOI.TerminationStatus()) == MOI.OPTIMAL || MOI.get(m, MOI.TerminationStatus()) == MOI.DUAL_INFEASIBLE
    Stacktrace:
     [1] [0m[1m(::Main.QuantumRelay.var"#scan#6"{Vector{JuMP.VariableRef}, Vector{Int64}, Vector{Int64}, Vector{Int64}, Int64, JuMP.Model})[22m[0m[1m([22m[90mc[39m::[0mChannel[90m{Any}[39m[0m[1m)[22m
    [90m   @ [39m[35mMain.QuantumRelay[39m [90m~/Downloads/[39m[90m[4mClp.jl:34[24m[39m
     [2] [0m[1m(::Base.var"#560#561"{Main.QuantumRelay.var"#scan#6"{Vector{JuMP.VariableRef}, Vector{Int64}, Vector{Int64}, Vector{Int64}, Int64, JuMP.Model}, Channel{Any}})[22m[0m[1m([22m[0m[1m)[22m
    [90m   @ [39m[90mBase[39m [90m./[39m[90m[4mchannels.jl:132[24m[39m

In [5]:
x=0.0 + 0.12008239133631671im
factorial(x)

LoadError: MethodError: no method matching factorial(::ComplexF64)
[0mClosest candidates are:
[0m  factorial([91m::Union{Int64, UInt64}[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/combinatorics.jl:27
[0m  factorial([91m::Union{Int16, Int32, Int8, UInt16, UInt32, UInt8}[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/combinatorics.jl:33
[0m  factorial([91m::SymPy.SymbolicObject[39m, [91m::Any...[39m; kwargs...) at ~/.julia/packages/SymPy/5GXQf/src/importexport.jl:42
[0m  ...

In [8]:
factorial(0.000234)

LoadError: MethodError: no method matching factorial(::Float64)
[0mClosest candidates are:
[0m  factorial([91m::Union{Int64, UInt64}[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/combinatorics.jl:27
[0m  factorial([91m::Union{Int16, Int32, Int8, UInt16, UInt32, UInt8}[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/combinatorics.jl:33
[0m  factorial([91m::SymPy.SymbolicObject[39m, [91m::Any...[39m; kwargs...) at ~/.julia/packages/SymPy/5GXQf/src/importexport.jl:42
[0m  ...